<a href="https://colab.research.google.com/github/jayantdahiya/Deep-Facial-Recognition/blob/main/Notebook1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [3]:
POS_PATH = os.path.join('/content/drive/MyDrive/Colab/Deep-Facial-Recognition-Application/data', '/content/drive/MyDrive/Colab/Deep-Facial-Recognition-Application/data/positive')
NEG_PATH = os.path.join('/content/drive/MyDrive/Colab/Deep-Facial-Recognition-Application/data', '/content/drive/MyDrive/Colab/Deep-Facial-Recognition-Application/data/negative')
ANC_PATH = os.path.join('/content/drive/MyDrive/Colab/Deep-Facial-Recognition-Application/data', '/content/drive/MyDrive/Colab/Deep-Facial-Recognition-Application/data/anchor')
# os.makedirs(POS_PATH)
# os.makedirs(NEG_PATH)
# os.makedirs(ANC_PATH)

In [6]:
#function to decode image with values between 0 and 1
def preprocess(file_path):
  byte_img = tf.io.read_file(file_path)
  img = tf.io.decode_jpeg(byte_img)
  img = tf.image.resize(img, (100,100))
  img = img/255.0
  return img

In [8]:
def preprocess_twin(input_img, validation_img, label):
  return(preprocess(input_img), preprocess(validation_img), label)

In [9]:
def make_embedding():
  inp = Input(shape=(100,100,3), name='input image')

  #First block
  c1 = Conv2D(64, (10,10), activation='relu')(inp)
  m1 = MaxPooling2D(64, (2,2), padding='same')(c1)

  #Second block
  c2 = Conv2D(128, (7,7), activation='relu')(m1)
  m2 = MaxPooling2D(64, (2,2), padding='same')(c2)

  #Third block
  c3 = Conv2D(128, (4,4), activation='relu')(m2)
  m3 = MaxPooling2D(64, (2,2), padding='same')(c3)

  #Final Embedding block
  c4 = Conv2D(256, (4,4), activation='relu')(m3)
  f1 = Flatten()(c4)
  d1 = Dense(4096, activation='sigmoid')(f1)

  return Model(inputs=[inp], outputs=[d1], name='embedding')

In [20]:
siamese_model = make_siamese_model()

In [14]:
embedding.summary()

Model: "embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input image (InputLayer)    [(None, 100, 100, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 91, 91, 64)        19264     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 46, 46, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 40, 40, 128)       401536    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 20, 20, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 17, 17, 128)       26

In [15]:
#Siamese L1 Distance class
class L1Dist(Layer):
  
  #Init method - inheritance
  def __init__(self, **kwargs):
    super().__init__()

  #Similarity calculation
  def call(self, input_embedding, validation_embedding):
    return tf.math.abs(input_embedding - validation_embedding)

In [16]:
l1 = L1Dist()

In [17]:
def make_siamese_model():

  #Anchor img input into the network
  input_img = Input(name='input_img', shape=(100,100,3))

  #Validation img in the network
  validation_img = Input(name='validation_img', shape=(100,100,3))

  #Combine siamese distance components
  siamese_layer = L1Dist()
  siamese_layer._name = 'distance'
  distances = siamese_layer(embedding(input_img), embedding(validation_img))

  #classification layer
  classifier = Dense(1, activation='sigmoid')(distances)

  return Model(inputs=[input_img, validation_img], outputs=classifier, name='Siamese_Network')

In [18]:
siamese_model = make_siamese_model()

In [21]:
model = tf.keras.models.load_model('/content/drive/MyDrive/Colab/Deep-Facial-Recognition-Application/siamesemodel.h5', custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [22]:
def verify(model, detection_threshold, verification_threshold):
    #build results array
    results = []
    for image in os.listdir(os.path.join('application_data','verification_images')):
        input_img = preprocess(os.path.join('application_data', 'input_images', 'input_image.jpg'))
        validation_img = preprocess(os.path.join('application_data','verification_images',image))
        
        #Make predictions
        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result)
        
    #Dectection threshold: Metric above which a prediction is considered positive
    detection = np.sum(np.array(results) > detection_threshold)
    #Verification threshold: Proportion of positive predictions/ total positive samples
    verification = detection/ len(os.listdir(os.path.join('application_data', 'verification_images')))
    verified = verification > verification_threshold
    
    return results, verified

In [23]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[420:420+250,500:500+250]
    
    cv2.imshow('Verification', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('v'):
        cv2.imwrite(os.path.join('application_data','input_images','input_image.jpg'), frame)
        results, verified = verify(model, 0.5, 0.5)
        print(verified)
        
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()